In [1]:
import math
import random

from pythreejs import *
from IPython.display import display

import numpy as np

In [2]:
scene = Scene(
    background='#000000'
)

camera = PerspectiveCamera(
    position=[10, 10, 10], 
    fov=50, 
    aspect=1.5
)

camera.lookAt([2, 1, 0])

renderer = Renderer(
    camera=camera, 
    scene=scene, 
    controls=[OrbitControls(controlling=camera)], 
    width=800, 
    height=600
)

In [3]:
def make_random_tracks(number_of_tracks):

    tracks = []
    
    for i in range(0,number_of_tracks+1):
        pt = 1.0 + random.random()*(50-2.0)
        charge = 1 if random.random() > 0.5 else -1
        eta = ((2*random.random())-1)*2.4
        phi = ((2*random.random())-1)*math.pi

        tracks.append(
            (pt,eta,phi,charge)
        )

    return tracks


def make_helix_points(track):

    pt = track[0]
    eta = track[1]
    phi = track[2]
    charge = track[3]
    
    vertex = (0,0,0)
                                                                                                                                                                                  
    Bz = 3.8
    radius = (pt / (math.fabs(charge)*Bz))

    centerX = vertex[0] + radius*charge*math.sin(phi)
    centerY = vertex[1] - radius*charge*math.cos(phi)
    
    pz = pt*math.sinh(eta)
    phi0 = math.atan2(vertex[1] - centerY, vertex[0] - centerX)
    omega = (charge*Bz) / pt
    pitch = pz / (pt*math.fabs(omega))

    maxAngle = 4*math.pi
    numPoints = 1000
    points = []

    for i in range(0, numPoints+1):

        t = (i/numPoints)*maxAngle
        x = centerX + radius*math.cos(phi0+t)
        y = centerY + radius*math.sin(phi0+t)
        z = vertex[2] + t*pitch
        r = math.sqrt(x*x +y*y)                                                                                                                                                                       
        
        if math.fabs(z) > 3 or r > 1.24:
            break
        
        points.append((float(x),float(y),float(z)))

    points = np.array(points)
    return points

def make_eb():

    geometry = CylinderGeometry(
        radiusTop=1.12, 
        radiusBottom=1.24, 
        height=6.0, 
        radialSegments=64, 
        heightSegments=1, 
        openEnded=True, 
        thetaStart=0, 
        thetaLength=2*math.pi
    )

    eb = Mesh(
        geometry,
        MeshBasicMaterial(
            color='#7fccff',
            wireframe=True,
            transparent=True,
            opacity=0.2
        )
    )

    eb.rotateX(math.pi/2)
    eb.name = 'EB'
    
    return eb

In [4]:
def add_axes():
    
    axes = []

    length = 1.0
    
    x_axis = ArrowHelper(
        dir=[1, 0, 0],
        origin=[0, 0, 0],
        length=length,
        color='#ff0000',
        headLength=0.2,
        headWidth=0.1
    )
    x_axis.name = 'X_Axis'
    axes.append(x_axis)
        
    y_axis = ArrowHelper(
        dir=[0, 1, 0],
        origin=[0, 0, 0],
        length=length,
        color='#00ff00',
        headLength=0.2,
        headWidth=0.1
    )
    y_axis.name = 'Y_Axis'
    axes.append(y_axis)
        
    z_axis = ArrowHelper(
        dir=[0, 0, 1],
        origin=[0, 0, 0],
        length=length,
        color='#0000ff',
        headLength=0.2,
        headWidth=0.1
    )
    z_axis.name = 'Z_Axis'
    axes.append(z_axis)
        
    return axes


In [5]:
scene.children = []

tracks = make_random_tracks(100)

for track in tracks:
    points = make_helix_points(track)
    points = np.array(points)

    geometry = BufferGeometry(attributes={
        'position': BufferAttribute(points, normalized=False)
    })
    
    material = LineBasicMaterial(color='#ffff00')
    
    line = Line(geometry, material)
    scene.add(line)

/Users/mccauley/ispy-nanoaod/ispy/lib/python3.11/site-packages/pythreejs/traits.py:257: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


In [6]:
scene.add(make_eb())
scene.add(add_axes())
display(renderer)

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 10.0, 10.0), projectionMatrix=(1.429671280339705…